# tsam - 1. Example
Example usage of the time series aggregation module (tsam)
Date: 02.05.2020

Author: Maximilian Hoffmann

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2
import copy
import os
import pandas as pd
import matplotlib.pyplot as plt
import tsam.timeseriesaggregation as tsam
%matplotlib inline

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv('testdata.csv', index_col = 0)

Show a slice of the dataset

In [ ]:
raw.head()

Show the shape of the raw input data: 4 types of timeseries (GHI, Temperature, Wind and Load) for every hour in a year

In [ ]:
raw.shape

Create a plot function for the temperature for a visual comparison of the time series

In [ ]:
def plotTS(data, periodlength, vmin, vmax):
    fig, axes = plt.subplots(figsize = [6, 2], dpi = 100, nrows = 1, ncols = 1)
    stacked, timeindex = tsam.unstackToPeriods(copy.deepcopy(data), periodlength)
    cax = axes.imshow(stacked.values.T, interpolation = 'nearest', vmin = vmin, vmax = vmax)
    axes.set_aspect('auto')  
    axes.set_ylabel('Hour')
    plt.xlabel('Day')

    fig.subplots_adjust(right = 1.2)
    cbar=plt.colorbar(cax)    
    cbar.set_label('T [°C]')

Plot an example series - in this case the temperature

In [ ]:
plotTS(raw['T'], 24, vmin = raw['T'].min(), vmax = raw['T'].max())

### Simple k-mean aggregation

Initialize an aggregation class object with k-mean as method for eight typical days, without any integration of extreme periods. Alternative clusterMethod's are 'averaging','hierarchical', 'k_medoids' and 'k_maxoids'.

In [ ]:
aggregation = tsam.TimeSeriesAggregation(raw, noTypicalPeriods = 8, hoursPerPeriod = 24, 
                                        clusterMethod = 'k_means')

Create the typical periods

In [ ]:
typPeriods = aggregation.createTypicalPeriods()

Show shape of typical periods: 4 types of timeseries for 8*24 hours

In [ ]:
typPeriods.shape

Save typical periods to .csv file

In [ ]:
typPeriods.to_csv(os.path.join('results','testperiods_kmeans.csv'))

Repredict the original time series based on the typical periods

In [ ]:
predictedPeriods = aggregation.predictOriginalData()

Plot the repredicted data

In [ ]:
plotTS(predictedPeriods['T'], 24, vmin = raw['T'].min(), vmax = raw['T'].max())

As seen, they days with the minimal temperature are excluded. In case that they are required they can be added to the aggregation as follow.

### k-maxoids aggregation including extreme periods

Initialize a time series aggregation based on k-maxoids, which automatically searches for points closest to the convex hull.

In [ ]:
aggregationMax = tsam.TimeSeriesAggregation(raw, noTypicalPeriods = 8, hoursPerPeriod = 24, 
                                            clusterMethod = 'k_maxoids', rescaleClusterPeriods=False)

Create the typical periods

In [ ]:
typPeriodsMax = aggregationMax.createTypicalPeriods()

The aggregation can also be evaluated by indicators

In [ ]:
aggregationMax.accuracyIndicators()

Repredict the original time series based on the typical periods

In [ ]:
predictedPeriodsMax = aggregationMax.predictOriginalData()

Plot repredicted data

In [ ]:
plotTS(predictedPeriodsMax['T'], 24, vmin = raw['T'].min(), vmax = raw['T'].max())

Here bigger biggest values and lower lowest values can be observed compared to k-means clustering.

### Comparison of the aggregations 
It was shown for the temperature, but both times all four time series have been aggregated. Therefore, we compare here also the duration curves  of the electrical load for the original time series, the aggregation with k-mean, and the k-maxoids aggregation.

In [ ]:
fig, axes = plt.subplots(figsize = [6, 2], dpi = 100, nrows = 1, ncols = 1)
raw['Load'].sort_values(ascending=False).reset_index(drop=True).plot(label = 'Original')
predictedPeriods['Load'].sort_values(ascending=False).reset_index(drop=True).plot(label = '8 typ days \n + (Centroids)')
predictedPeriodsMax['Load'].sort_values(
    ascending=False).reset_index(drop=True).plot(label = '8 typ days \n + (Maxoids)')
plt.legend()
plt.xlabel('Hours [h]')
plt.ylabel('Duration Load [MW]')

Or as unsorted time series for an example week

In [ ]:
fig, axes = plt.subplots(figsize = [6, 2], dpi = 100, nrows = 1, ncols = 1)
raw['Load']['20100210':'20100218'].plot(label = 'Original')
predictedPeriods['Load']['20100210':'20100218'].plot(label = '8 typ days \n + (Centroids)')
predictedPeriodsMax['Load']['20100210':'20100218'].plot(label = '8 typ days \n + (Maxoids)')
plt.legend()
plt.ylabel('Load [MW]')